In [1]:
using JuMP, Cbc
using Combinatorics

In [2]:
function happiness(table):
    """
    Find the happiness of the table
    - by calculating the maximum distance between the letters
    """
    return abs(Int(table[1][1]) - Int(table[end][1]))
end

happiness (generic function with 1 method)

In [85]:
max_tables = 5
max_table_size = 4
guests = ["A" "B" "C" "D" "E" "F" "G" "I" "J" "K" "L" "M" "N" "O" "P" "Q" "R"]

1×17 Array{String,2}:
 "A"  "B"  "C"  "D"  "E"  "F"  "G"  "I"  …  "L"  "M"  "N"  "O"  "P"  "Q"  "R"

In [87]:
length(guests)

17

In [88]:
table_combos = [collect(combinations(guests, t)) for t in 1:max_table_size]
;

In [89]:
# this seems suboptimal, there's probably a better to flatten out these combinations
possible_tables = []
for c in table_combos
    for t in c
        append!(possible_tables, [t])
    end
end

In [90]:
m = Model(solver=CbcSolver())
num_possible_tables = length(possible_tables)
idx_possible_tables = 1:num_possible_tables

@variable(m, table_assignment[idx_possible_tables], Bin)

# Objective: maximize happiness = minimize happiness value
@objective(m, Min, sum([happiness(possible_tables[t]) * table_assignment[t] for t in idx_possible_tables]))

@constraint(m, sum([table_assignment[t] for t in idx_possible_tables]) <= max_tables)

for guest in guests
    @constraint(m, sum([table_assignment[t] for t in idx_possible_tables if guest in possible_tables[t]]) == 1)
end

;


In [91]:
status = solve(m)

println("Objective value: ", getobjectivevalue(m))

Objective value: 12.0


In [92]:
table_assignment

table_assignment[i] ∈ {0,1} ∀ i ∈ {1,2,…,3212,3213}

In [93]:
# println("Solution:  ", getvalue(table_assignment))

In [96]:
for i=1:length(table_assignment)
    if getvalue(table_assignment[i]) == 1
        println(possible_tables[i], " - ", happiness(possible_tables[i]))
    end
end

String["Q", "R"] - 1
String["E", "F", "G"] - 2
String["A", "B", "C", "D"] - 3
String["I", "J", "K", "L"] - 3
String["M", "N", "O", "P"] - 3
